In [14]:
import numpy as np
from reactionmodel.model import Species, Reaction, Model

In [46]:
X = Species('X')

schlogl_reactions = [
    Reaction([(X,2)], [(X,3)], k='c1*N1'),
    Reaction([(X,3)], [(X,2)], k='c2'),
    Reaction([], [X], k='c3*N2'),
    Reaction([X], [], k='c4')
]

schlogl_params = {
    'c1': 3e-7,
    'c2': 1e-4,
    'c3': 1e-3,
    'c4': 3.5,
    'N1': 1e5,
    'N2': 2e5,
}

schlogl = Model([X], schlogl_reactions)

A = Species('A')
B = Species('B')

background_reactions = [
    Reaction([A], [B], k='k1'),
    Reaction([B], [A], k='k2')
]

background_params = {
    'k1': 2e7,
    'k2': 1e7,
}

background = Model([A,B], background_reactions)

combined = Model([A,B,X], schlogl_reactions + background_reactions)
combined_params = background_params.copy()
combined_params.update(schlogl_params)

In [47]:
combined_params

{'k1': 20000000.0,
 'k2': 10000000.0,
 'c1': 3e-07,
 'c2': 0.0001,
 'c3': 0.001,
 'c4': 3.5,
 'N1': 100000.0,
 'N2': 200000.0}

In [48]:
from hybrid.tau import TauLeapSimulator

tau_schlogl = TauLeapSimulator(
    schlogl.get_k(parameters=schlogl_params),
    schlogl.stoichiometry(),
    schlogl.kinetic_order(),
)

tau_background = TauLeapSimulator(
    background.get_k(parameters=background_params),
    background.stoichiometry(),
    background.kinetic_order(),
)

tau_combined = TauLeapSimulator(
    combined.get_k(parameters=combined_params),
    combined.stoichiometry(),
    combined.kinetic_order(),
)

Evaluating expression: c1*N1 => 0.03
Evaluating expression: c2 => 0.0001
Evaluating expression: c3*N2 => 200.0
Evaluating expression: c4 => 3.5
Evaluating expression: k1 => 20000000.0
Evaluating expression: k2 => 10000000.0
Evaluating expression: c1*N1 => 0.03
Evaluating expression: c2 => 0.0001
Evaluating expression: c3*N2 => 200.0
Evaluating expression: c4 => 3.5
Evaluating expression: k1 => 20000000.0
Evaluating expression: k2 => 10000000.0


In [49]:
tau_schlogl.simulate(
    [0.0, 10.0],
    schlogl.make_initial_condition({'X':250}),
    np.random.default_rng()
)

History(t=10.0, y=array([104.]), t_history=array([0.00000000e+00, 1.86562582e-05, 1.66803249e-04, ...,
       9.99801508e+00, 9.99952141e+00, 1.00000000e+01]), y_history=array([[250., 249., 250., ..., 103., 104., 104.]]), step_indices=array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 1.1885e+04, 1.1886e+04,
       1.1887e+04]), status_history=array([nan,  3.,  3., ...,  3.,  3.,  2.]), pathway_history=array([inf, inf, inf, ..., inf, inf, inf]), status_counter=Counter({<TauStepStatus.gillespie_stochastic: 3>: 11886, <TauStepStatus.rejected_for_gillespie: -2>: 119, <TauStepStatus.gillespie_t_end: 2>: 1}), pathway_counter=Counter({inf: 11888}))

In [50]:
tau_background.simulate(
    [0.0, 10.0],
    background.make_initial_condition({'A': 1e5, 'B': 2e5}),
    np.random.default_rng()
).plot(background.legend())

IndexError: index 1000000 is out of bounds for axis 0 with size 1000000

In [43]:
r = tau_combined.simulate(
    [0.0, 10.0],
    combined.make_initial_condition({'X': 250, 'A': 1e5, 'B': 1e5}),
    np.random.default_rng()
)

In [45]:
r.status_counter

Counter({<TauStepStatus.leap: 0>: 124729})